<a href="https://colab.research.google.com/github/Franosei/LLM_Meets_Football/blob/main/codes/rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import os
import nest_asyncio
nest_asyncio.apply()
from llama_parse import LlamaParse 
from llama_index.core import SimpleDirectoryReader
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors
import PyPDF2


In [ ]:
access_token = os.environ.get('HF_TOKEN')
api_passer = os.environ.get("LLAMA_CLOUD_API_KEY")

In [ ]:
country_list = [
    "Germany",
    "Albania",
    "Austria",
    "Belgium",
    "Croatia",
    "Czech Republic",
    "Denmark",
    "England",
    "France",
    "Hungary",
    "Italy",
    "Netherlands",
    "Portugal",
    "Romania",
    "Scotland",
    "Serbia",
    "Slovakia",
    "Slovenia",
    "Spain",
    "Switzerland",
    "Turkey"
]

In [ ]:
df1 = pd.read_csv("../Data/results.csv")
pattern = "|".join(country_list)

# Apply the pattern to both columns and keep rows where at least one column matches
filtered_df = df1[df1["home_team"].str.contains(pattern) | df1["away_team"].str.contains(pattern)]

In [ ]:
# Creating the sentence
sentences = []
for _, row in filtered_df.iterrows():
    sentence = f"On {row['date']}, {row['home_team']} played against {row['away_team']} in {row['city']}, {row['country']}. The match was a {row['tournament']} and was{' not' if row['neutral'] else ''} played on neutral ground. The home team {row['home_team']} scored {row['home_score']} goals, while the away team {row['away_team']} scored {row['away_score']} goals."
    sentences.append(sentence)

sentences_df = pd.DataFrame(sentences, columns=['sentence'])
sentences_df.to_csv("../Data/sentence.csv",index=False)

In [78]:
# Converting csv file to pdf

def csv_to_pdf(csv_file, pdf_file):
    data = pd.read_csv(csv_file)

    c = canvas.Canvas(pdf_file, pagesize=letter)
    width, height = letter 
    margin = 50
    content_width = width - 2 * margin
    text_height = 12
    line_height = 14
    
    def create_new_page():
        c.showPage()  
        c.setFont("Helvetica", 9) 
        c.drawString(margin, height - 40, "")
        c.setFont("Helvetica", 9)  
        c.drawString(margin, height - 60, ' | '.join(data.columns))
        return height - 74 

    c.setFont("Helvetica", 9)
    y = height - 60
    
    # Add column names as a header
    c.setFont("Helvetica", 9)
    c.drawString(margin, y, ' | '.join(data.columns))
    y -= line_height
    c.setFont("Helvetica", 9)
    
    # Add rows from the DataFrame
    for index, row in data.iterrows():
        row_data = ' | '.join(str(x) for x in row)
        row_width = c.stringWidth(row_data, "Helvetica", 9)
        if row_width > content_width:
            words = row_data.split()
            wrapped_text = ""
            current_line = ""
            for word in words:
                test_line = current_line + word + " "
                test_line_width = c.stringWidth(test_line, "Helvetica", 9)
                if test_line_width < content_width:
                    current_line = test_line
                else:
                    wrapped_text += current_line.rstrip() + "|"
                    current_line = word + " "
            wrapped_text += current_line
            for line in wrapped_text.split('|'):
                if y < margin + line_height:
                    y = create_new_page()
                c.drawString(margin, y, line)
                y -= line_height
        else:
            if y < margin + line_height:
                y = create_new_page()
            c.drawString(margin, y, row_data)
            y -= line_height
    
    c.save()

csv_file = '../Data/sentence.csv'
pdf_file = '../Data/output.pdf'

csv_to_pdf(csv_file, pdf_file)


In [ ]:
# Merging two pdfs 

def merge_pdfs(paths, output='../Data/merged_file.pdf'):
    pdf_writer = PyPDF2.PdfWriter()

    # Loop through all the PDF files
    for path in paths:
        pdf_reader = PyPDF2.PdfReader(path)
        # Add each page to the writer object
        for page in range(len(pdf_reader.pages)):
            pdf_writer.add_page(pdf_reader.pages[page])

    # Write out the merged PDF to a file named 'euro_info.pdf'
    with open(output, 'wb') as out:
        pdf_writer.write(out)

if __name__ == "__main__":
    pdf_paths = ['../Data/output.pdf', '../Data/Eurosdata.pdf']

    merge_pdfs(pdf_paths)

pdf_paths = ['../Data/output.pdf', '../Data/Eurosdata.pdf']
output_path = '../Data/merged_file.pdf'

merge_pdfs(pdf_paths, output_path)


In [26]:
parser = LlamaParse(
    api_key = api_passer,
    result_type="markdown"
)

In [27]:
documents = await parser.aload_data('../Data/Eurosdata.pdf')

Started parsing the file under job_id 63cc9dad-5b06-4ed1-9021-baaee592eba4


,sentence
0,"On 1872-11-30, Scotland played against England..."
1,"On 1873-03-08, England played against Scotland..."
2,"On 1874-03-07, Scotland played against England..."
3,"On 1875-03-06, England played against Scotland..."
4,"On 1876-03-04, Scotland played against England..."
...,...
11281,"On 2024-03-26, Norway played against Slovakia ..."
11282,"On 2024-03-26, Republic of Ireland played agai..."
11283,"On 2024-03-26, Scotland played against Norther..."
11284,"On 2024-03-26, Slovenia played against Portuga..."


In [79]:
sentences_df.to_csv("../Data/sentence.csv",index=False)